In [1]:
import tools
import param
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [2]:
def dropTipToWaste(tool, x_slot, y_slot, z):
    tool.robot.move(z=z)
    x, y = param.calcSquareSlotCenterFromVertices(x_slot, y_slot)
    tool.robot.move(x=x, y=y)
    tool.dropTip()

In [2]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10']

In [3]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [4]:
ar.home()

In [5]:
stp = tools.stationary_touch_probe(ar)

In [6]:
p1000_tip_rack = racks.consumables(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [16]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
magep1 = racks.rack(rack_name='eppendorf_magnetic_1', rack_type='eppendorf_magnetic')
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [17]:
sample_list = [samples.sample(sample_name='t_ep_'+str(x), sample_type='eppendorf_tube') for x in range(5)]
[x[0].place(magep1, x[1], 1) for x in zip(sample_list, range(5))]

[None, None, None, None, None]

In [10]:
ethanol = samples.sample(sample_name='ethanol', sample_type='50mL_tube')
ethanol.place(r50, 2, 0)
ethanol.setVolume(50000)

In [11]:
# Waste tube
waste = samples.sample(sample_name='waste', sample_type='50mL_tube')
waste.place(r50, 1, 0)

In [12]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [13]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [18]:
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)
ethanol_vol_list = [200 for x in sample_list]
timestamp = time.time()
p1000.distributeLiquid(ethanol, sample_list, ethanol_vol_list, raise_z=300)
dropTipToWaste(tool=p1000, x_slot=5, y_slot=3, z=300)

In [22]:
p1000.returnTool()

In [17]:
p1000.movePlunger(0)

In [18]:
dropTipToWaste(tool=p1000, x_slot=5, y_slot=3, z=300)

In [23]:
ar.home()

In [26]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [27]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [19]:
for sample in sample_list:
    xn, yn = p1000_tip_rack.getNextConsumable()
    p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)
    p1000.uptakeAll(sample)
    p1000.dispenseLiquid(waste, 1000, blow_extra=True)
    dropTipToWaste(tool=p1000, x_slot=5, y_slot=3, z=300)

In [20]:
p1000.returnTool()

In [21]:
ar.home()

# Introducing a waste rack

In [6]:
waste_rack = racks.rack(rack_name='waste', rack_type='waste_rack', x_slot=5, y_slot=3)

In [7]:
waste_rack.rack_data

{'name': 'waste', 'n_x': 5, 'n_y': 3, 'type': 'waste_rack'}

In [8]:
tp = tools.mobile_touch_probe.getTool(ar)

In [9]:
x, y, z = calibration.calibrateStationaryProbe(tp, stp)

In [10]:
ar.move(z=300)

In [11]:
xr, yr, zr = calibration.calibrateRack(tp, waste_rack)

In [12]:
waste_rack.rack_data

{'name': 'waste',
 'n_x': 5,
 'n_y': 3,
 'type': 'waste_rack',
 'position': [835.99, 394.35, 509.05],
 'pos_stalagmyte': [93.4, 65.85, 490.15]}

In [13]:
tp.returnTool()

# Testing tip to waste dropping

In [7]:
p1000_tip_rack = racks.consumables(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [8]:
waste_rack = racks.rack(rack_name='waste', rack_type='waste_rack')

In [9]:
p1000 = tools.pipettor.getTool(robot=ar, tool_name='p1000_tool')

In [10]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [11]:
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)

In [12]:
p1000.dropTipToWaste(waste_rack, raise_z=400)

In [13]:
p1000.returnTool()

In [14]:
ar.home()